In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [84]:
df = pd.read_csv('data_with_dists.csv', sep=';')

In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1130 entries, 0 to 1129
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   number of rooms       1130 non-null   int64  
 1   area of apartment     1130 non-null   float64
 2   number of floors      1130 non-null   int64  
 3   apartment floor       1130 non-null   int64  
 4   price                 1130 non-null   float64
 5   repair                1130 non-null   object 
 6   bathroom              1130 non-null   object 
 7   terrace               1130 non-null   object 
 8   year of construction  1130 non-null   int64  
 9   elevator              1130 non-null   int64  
 10  garbage chute         1130 non-null   int64  
 11  type of house         1130 non-null   object 
 12  district              1130 non-null   object 
 13  street                1130 non-null   object 
 14  concierge             1130 non-null   int64  
 15  distance             

## Удаление дорогих квартир

In [86]:
percentile_82 = np.percentile(df['price'], 82)
print(percentile_82)

5900000.0


In [87]:
df = df[df['price'] < percentile_82]
df.index = range(len(df))

# Районы

In [88]:
df.district.unique()

array(['р-н Индустриальный', 'р-н Свердловский', 'р-н Мотовилихинский',
       'р-н Орджоникидзевский', 'р-н Дзержинский', 'р-н Кировский',
       'р-н Ленинский'], dtype=object)

### Добавление для каждого района средней цены за квадратный метр

In [89]:
district_features = dict()
for district in df.district.unique():
    district_features[district] = []

In [90]:
df['price_for_metre'] = np.array([df['price'][i] / df['area of apartment'][i] for i in range(len(df))])

In [91]:
for district in district_features:
    district_features[district].append(df.groupby("district")["price_for_metre"].mean()[district])
df = df.drop("price_for_metre", axis=1)

In [92]:
district_features

{'р-н Индустриальный': [78168.86137903988],
 'р-н Свердловский': [74318.1778421745],
 'р-н Мотовилихинский': [69325.50503743357],
 'р-н Орджоникидзевский': [53809.059009550336],
 'р-н Дзержинский': [76055.00621772064],
 'р-н Кировский': [62889.363336951275],
 'р-н Ленинский': [85854.45308674303]}

### Добавление населения для каждого района

https://ru.wikipedia.org/wiki/Административное_деление_Перми

In [93]:
for district in district_features:
    print(district)
    population = int(input())
    district_features[district].append(population)

р-н Индустриальный
169613
р-н Свердловский
218220
р-н Мотовилихинский
193639
р-н Орджоникидзевский
115722
р-н Дзержинский
166466
р-н Кировский
130798
р-н Ленинский
54741


In [94]:
district_features

{'р-н Индустриальный': [78168.86137903988, 169613],
 'р-н Свердловский': [74318.1778421745, 218220],
 'р-н Мотовилихинский': [69325.50503743357, 193639],
 'р-н Орджоникидзевский': [53809.059009550336, 115722],
 'р-н Дзержинский': [76055.00621772064, 166466],
 'р-н Кировский': [62889.363336951275, 130798],
 'р-н Ленинский': [85854.45308674303, 54741]}

### Добавление экологической обстановки, уровня озеленения для каждого района

https://markakachestva.ru/amenities/5494-luchshie-rajony-permi-dlja-prozhivanija-rejting.html

хорошая обстановка, достаточно озеленения - 4

благоприятная экологическая обстановка, обильное озеленение - 4

отличная обстановка, ¾ района – парки и лесопарковые зоны - 5
 
удовлетворительная обстановка, достаточный уровень озеленения - 3
  
много парковых зон, Черняевский лес - 5
  
хорошая обстановка, достаточное озеленение - 4
   
удовлетворительная обстановка, скудное озеленение - 2

In [95]:
for district in district_features:
    print(district)
    rate = int(input())
    district_features[district].append(rate)

р-н Индустриальный
3
р-н Свердловский
2
р-н Мотовилихинский
4
р-н Орджоникидзевский
4
р-н Дзержинский
5
р-н Кировский
5
р-н Ленинский
4


In [96]:
district_features

{'р-н Индустриальный': [78168.86137903988, 169613, 3],
 'р-н Свердловский': [74318.1778421745, 218220, 2],
 'р-н Мотовилихинский': [69325.50503743357, 193639, 4],
 'р-н Орджоникидзевский': [53809.059009550336, 115722, 4],
 'р-н Дзержинский': [76055.00621772064, 166466, 5],
 'р-н Кировский': [62889.363336951275, 130798, 5],
 'р-н Ленинский': [85854.45308674303, 54741, 4]}

### Добавление в датасет

In [97]:
df['price for metre'] = np.array([district_features[df['district'][index]][0] for index in range(len(df))])

In [98]:
df['district population'] = np.array([district_features[df['district'][index]][1] for index in range(len(df))])

In [99]:
df['green rate'] = np.array([district_features[df['district'][index]][2] for index in range(len(df))])

In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 925 entries, 0 to 924
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   number of rooms       925 non-null    int64  
 1   area of apartment     925 non-null    float64
 2   number of floors      925 non-null    int64  
 3   apartment floor       925 non-null    int64  
 4   price                 925 non-null    float64
 5   repair                925 non-null    object 
 6   bathroom              925 non-null    object 
 7   terrace               925 non-null    object 
 8   year of construction  925 non-null    int64  
 9   elevator              925 non-null    int64  
 10  garbage chute         925 non-null    int64  
 11  type of house         925 non-null    object 
 12  district              925 non-null    object 
 13  street                925 non-null    object 
 14  concierge             925 non-null    int64  
 15  distance              9

# Ремонт

Оценка ремонта

In [101]:
df['repair'].unique()

array(['косметический', 'евро', 'требует ремонта', 'дизайнерский'],
      dtype=object)

In [102]:
df.groupby(by="repair")["price"].mean()

repair
дизайнерский       4.004933e+06
евро               3.766200e+06
косметический      3.340751e+06
требует ремонта    3.107113e+06
Name: price, dtype: float64

In [103]:
def get_repair_rate(repair):
    if repair == 'требует ремонта':
        return 0
    elif repair == 'косметический':
        return 1
    elif repair == 'евро':
        return 2
    else:
        return 3

In [104]:
df['repair rate'] = np.array([get_repair_rate(df['repair'][i]) for i in range(len(df))])

# Ванная

In [105]:
df['bathroom'].unique()

array(['раздельный', 'совмещенный', 'несколько'], dtype=object)

In [106]:
df.groupby(by="bathroom")["price"].mean()

bathroom
несколько      5.443000e+06
раздельный     3.688366e+06
совмещенный    3.110461e+06
Name: price, dtype: float64

In [107]:
df.groupby(by="bathroom")["price"].count()

bathroom
несколько       10
раздельный     457
совмещенный    458
Name: price, dtype: int64

In [108]:
def get_bathroom_rate(bathroom):
    if bathroom == 'совмещенный':
        return 1
    elif bathroom == 'раздельный':
        return 2
    else:
        return 6

In [109]:
df['bathroom rate'] = np.array([get_bathroom_rate(df['bathroom'][i]) for i in range(len(df))])

# Балкон

In [110]:
df['terrace'].unique()

array(['балкон', 'нет', 'лоджия'], dtype=object)

In [111]:
df.groupby(by="terrace")["price"].mean()

terrace
балкон    3.238130e+06
лоджия    3.848862e+06
нет       2.503152e+06
Name: price, dtype: float64

In [112]:
df.groupby(by="terrace")["price"].count()

terrace
балкон    304
лоджия    465
нет       156
Name: price, dtype: int64

In [113]:
def get_terrace_rate(terrace):
    if terrace == 'нет':
        return 0
    elif terrace == 'балкон':
        return 1
    else:
        return 2

In [114]:
df['terrace rate'] = np.array([get_terrace_rate(df['terrace'][i]) for i in range(len(df))])

## Год конструкции

In [115]:
df['age'] = 2021 - df['year of construction']

# Сохранение

In [116]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 925 entries, 0 to 924
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   number of rooms       925 non-null    int64  
 1   area of apartment     925 non-null    float64
 2   number of floors      925 non-null    int64  
 3   apartment floor       925 non-null    int64  
 4   price                 925 non-null    float64
 5   repair                925 non-null    object 
 6   bathroom              925 non-null    object 
 7   terrace               925 non-null    object 
 8   year of construction  925 non-null    int64  
 9   elevator              925 non-null    int64  
 10  garbage chute         925 non-null    int64  
 11  type of house         925 non-null    object 
 12  district              925 non-null    object 
 13  street                925 non-null    object 
 14  concierge             925 non-null    int64  
 15  distance              9

In [117]:
df.to_csv('data_with_new_features.csv', sep=';', index=False)